In [1]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS

In [2]:
g = rdflib.ConjunctiveGraph()

# parse a local RDF file by specifying the format
result = g.parse("artchives_birthplaces.nq", format='nquads')

In [3]:
wd = Namespace("http://www.wikidata.org/entity/") # remember that a prefix matches a URI until the last slash (or hashtag #)
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")

In [4]:
# Get the list of art historians in our graph "g"
arthistorians_list = set()

# iterate over the triples in the graph
for s,p,o in g.triples(( None, wdt.P170, None)):   
    if "wikidata.org/entity/" in str(o):           
        arthistorians_list.add('<' + str(o) + '>')     
    
print(arthistorians_list)

{'<http://www.wikidata.org/entity/Q1089074>', '<http://www.wikidata.org/entity/Q90407>', '<http://www.wikidata.org/entity/Q1296486>', '<http://www.wikidata.org/entity/Q6700132>', '<http://www.wikidata.org/entity/Q1715096>', '<http://www.wikidata.org/entity/Q457739>', '<http://www.wikidata.org/entity/Q60185>', '<http://www.wikidata.org/entity/Q1373290>', '<http://www.wikidata.org/entity/Q18935222>', '<http://www.wikidata.org/entity/Q55453618>', '<http://www.wikidata.org/entity/Q2824734>', '<http://www.wikidata.org/entity/Q1271052>', '<http://www.wikidata.org/entity/Q1641821>', '<http://www.wikidata.org/entity/Q19997512>', '<http://www.wikidata.org/entity/Q61913691>', '<http://www.wikidata.org/entity/Q995470>', '<http://www.wikidata.org/entity/Q1629748>', '<http://www.wikidata.org/entity/Q85761254>', '<http://www.wikidata.org/entity/Q3051533>', '<http://www.wikidata.org/entity/Q41616785>', '<http://www.wikidata.org/entity/Q3057287>', '<http://www.wikidata.org/entity/Q537874>', '<http://w

In [181]:
#volevo ottenere solo i labels degli storici ma qualcosa è sbagliato
from rdflib.namespace import RDFS
AH = set()

for s,p,o in g.triples((wd.Q5, wdt.P170, None)):
    for s1,p1,o1 in g.triples((RDFS.label, p, None)):
        AH.add(s1.strip())
        
print(AH)

set()


# Inizio step 1

In [147]:

import pandas as pd
# create first dataframe only using the specified columns 
data = pd.read_csv("DoAH_StoriciItaliani_csvseparatodavirgole.csv",
                    usecols=["Full Name", "Gender", "Archives"])
# axis 0 to drop the rows, subset to only remove NaNs from the column Archives
data.dropna(axis=0, subset=["Archives"], inplace=True)
data.drop_duplicates()
data.reset_index(inplace=True, drop=True)
#data["Archives"] = data["Archives"].apply(lambda x: x.strip(" ,.").split(","))
data.to_pickle("database_degli_eroi.pickle")
data

,Full Name,Gender,Archives
0,"Accascina, Maria",female,"Comune di Palermo Concetta Di Natale, Maria Ma..."
1,"Argan, Giulio Carlo",male,Private Archive in Rome.
2,"Bellosi, Luciano",male,"\n\tArchivio Lucino Bellosi, Biblioteca Umanis..."
3,"Bertini Calosso, Achille",male,"Fondo Bertini Calosso, Istituto Nazionale di A..."
4,"Bianchi Bandinelli, Ranuccio",male,"Ranuccio Bianchi Bandinelli, lâ€™Archivio di S..."
5,"Boni, Giacomo",male,"Boni-Tea Archive, Istituto lombardo - Accademi..."
6,"Bossi, Giuseppe",male,"Il Fondo Giuseppe Bossi, Biblioteca Ambrosiana..."
7,"Brizio, Edoardo",male,"Archivio Edoardo Brizio, Biblioteca comunale d..."
8,"Calvi, Girolamo-Luigi",male,"Carte di Girolamo Luigi Calvi, UniversitÃ deg..."
9,"Carli, Enzo",male,"Carli Enzo, Biblioteca Umanistica dellâ€™Unive..."


# To resolve:
0. utf-8 chars are not always displayed correctly: fix with regex?
1. full names are reversed (`surname, name`)
2. we need to have a controlled entity (`wd:xyz`) for each name, to be able to link them to other info
3. archives are a list of strings 

In [148]:
def reformat_names(name):
    """ reverse names from surname,name format to name surname """
    l = name.split(", ")
    new = " ".join(reversed(l))
    return new

In [149]:
# reverse names and remove duplicate whitespace
data["Full Name"] = data["Full Name"].apply(reformat_names)
data["Full Name"] = data["Full Name"].apply(lambda x: x.replace("  ", " "))
data["Full Name"] = data["Full Name"].apply(lambda x: x.replace("-", " "))

data.head()

,Full Name,Gender,Archives
0,Maria Accascina,female,"Comune di Palermo Concetta Di Natale, Maria Ma..."
1,Giulio Carlo Argan,male,Private Archive in Rome.
2,Luciano Bellosi,male,"\n\tArchivio Lucino Bellosi, Biblioteca Umanis..."
3,Achille Bertini Calosso,male,"Fondo Bertini Calosso, Istituto Nazionale di A..."
4,Ranuccio Bianchi Bandinelli,male,"Ranuccio Bianchi Bandinelli, lâ€™Archivio di S..."


In [150]:
from qwikidata.sparql import return_sparql_query_results

In [151]:
historian_entity_from_label = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?artHistorian ?o
WHERE {{
    OPTIONAL{{
        ?artHistorian wdt:P106 wd:Q1792450 .
    }}
    ?artHistorian wdt:P31 wd:Q5 .
    ?artHistorian rdfs:label ?o 
                  FILTER ( str(?o) = "{}" )  
                  FILTER ( lang(?o) = "it" ).
}}
"""

In [152]:
def find_historian_entity_from_name(name: str):
    if name.startswith("conte"):
        name = name.lstrip("conte ")
    query = historian_entity_from_label.format(name)
    res = return_sparql_query_results(query_string=query)
    try:
        wdt_uri = res['results']['bindings'][0]['artHistorian']['value']
    except (IndexError, KeyError):
        return ""
    return wdt_uri.split("/")[-1]

In [153]:
data["Historian Entity"] = data["Full Name"].apply(find_historian_entity_from_name)
data.to_pickle("database_degli_eroi.pickle")
data.head()

,Full Name,Gender,Archives,Historian Entity
0,Maria Accascina,female,"Comune di Palermo Concetta Di Natale, Maria Ma...",Q98804253
1,Giulio Carlo Argan,male,Private Archive in Rome.,Q778445
2,Luciano Bellosi,male,"\n\tArchivio Lucino Bellosi, Biblioteca Umanis...",Q1058859
3,Achille Bertini Calosso,male,"Fondo Bertini Calosso, Istituto Nazionale di A...",Q19753744
4,Ranuccio Bianchi Bandinelli,male,"Ranuccio Bianchi Bandinelli, lâ€™Archivio di S...",Q471179


In [154]:
data

,Full Name,Gender,Archives,Historian Entity
0,Maria Accascina,female,"Comune di Palermo Concetta Di Natale, Maria Ma...",Q98804253
1,Giulio Carlo Argan,male,Private Archive in Rome.,Q778445
2,Luciano Bellosi,male,"\n\tArchivio Lucino Bellosi, Biblioteca Umanis...",Q1058859
3,Achille Bertini Calosso,male,"Fondo Bertini Calosso, Istituto Nazionale di A...",Q19753744
4,Ranuccio Bianchi Bandinelli,male,"Ranuccio Bianchi Bandinelli, lâ€™Archivio di S...",Q471179
5,Giacomo Boni,male,"Boni-Tea Archive, Istituto lombardo - Accademi...",Q572091
6,Giuseppe Bossi,male,"Il Fondo Giuseppe Bossi, Biblioteca Ambrosiana...",Q428638
7,Edoardo Brizio,male,"Archivio Edoardo Brizio, Biblioteca comunale d...",
8,Girolamo Luigi Calvi,male,"Carte di Girolamo Luigi Calvi, UniversitÃ deg...",Q3768943
9,Enzo Carli,male,"Carli Enzo, Biblioteca Umanistica dellâ€™Unive...",Q3055532


In [184]:
#see all the rows 

data_by_historian = data.groupby( ["Title", "Archives"] ).size().reset_index()
data_by_historian.columns = ["Title", "Archives", "Overview"]
# see the first rows
pd.set_option('display.max_rows', data.shape[0]+1)
print(data_by_historian)

                           Title  \
0               Accascina, Maria   
1            Argan, Giulio Carlo   
2               Bellosi, Luciano   
3       Bertini Calosso, Achille   
4   Bianchi Bandinelli, Ranuccio   
5                  Boni, Giacomo   
6                Bossi, Giuseppe   
7                Brizio, Edoardo   
8          Calvi, Girolamo-Luigi   
9                    Carli, Enzo   
10                Carocci, Guido   
11       Castiglione, Baldassare   
12                Cecchi, Emilio   
13       Cesnola, Luigi Palma di   
14    Cicognara, Leopoldo, Conte   
15                Coletti, Luigi   
16              Condivi, Ascanio   
17              Croce, Benedetto   
18               D'Ancona, Paolo   
19          De Vecchi, Pierluigi   
20                   Ragusa, Isa   
21                Salerno, Luigi   
22                Zeri, Federico   
23                   Zevi, Bruno   

                                             Archives  Overview  
0   Comune di Palermo Concetta Di

In [183]:
unique_dictionaryofArtHistorians = data_by_historian["Title"].unique()
print(unique_dictionaryofArtHistorians)

['Accascina, Maria' 'Argan, Giulio Carlo' 'Bellosi, Luciano'
 'Bertini Calosso, Achille' 'Bianchi Bandinelli, Ranuccio' 'Boni, Giacomo'
 'Bossi, Giuseppe' 'Brizio, Edoardo' 'Calvi, Girolamo-Luigi' 'Carli, Enzo'
 'Carocci, Guido' 'Castiglione, Baldassare' 'Cecchi, Emilio'
 'Cesnola, Luigi Palma di' 'Cicognara, Leopoldo, Conte' 'Coletti, Luigi'
 'Condivi, Ascanio' 'Croce, Benedetto' "D'Ancona, Paolo"
 'De Vecchi, Pierluigi' 'Ragusa, Isa' 'Salerno, Luigi' 'Zeri, Federico'
 'Zevi, Bruno']


In [171]:
#raddrizzo i nomi
cleanHistorianSet = set()

for i in unique_dictionaryofArtHistorians:
    nocomma = i.replace(",", "")
    splittiamo = nocomma.split(" ")
    invertiamonome_cognome = list(reversed(splittiamo))
    join = ' '.join(invertiamonome_cognome)
    cleanHistorianSet.add(join)
    
    print(join)

Maria Accascina
Carlo Giulio Argan
Luciano Bellosi
Achille Calosso Bertini
Ranuccio Bandinelli Bianchi
Giacomo Boni
Giuseppe Bossi
Edoardo Brizio
Girolamo-Luigi Calvi
Enzo Carli
Guido Carocci
Baldassare Castiglione
Emilio Cecchi
di Palma Luigi Cesnola
Conte Leopoldo Cicognara
Luigi Coletti
Ascanio Condivi
Benedetto Croce
Paolo D'Ancona
Pierluigi Vecchi De
Isa Ragusa
Luigi Salerno
Federico Zeri
Bruno Zevi
